# Intro to Pachyderm Tutorial
<img src="images/nb_header.png" alt="Drawing" style="width: 600px;"/>

### Welcome to Pachyderm! 

Pachyderm is an incredibly powerful platform, and can be used for many kinds of data-centered applications. In this notebook, we will introduce you to the basic concepts of data versioning and data pipelines and how they work in Pachyderm. 

## Installation

For this tutorial, we will use the `pachctl` command line interface. This means that any of these commands can be run from your terminal as well. 

If you are running this notebook on [Pachyderm Hub](https://hub.pachyderm.com/), the installation is done for you and everything should work automatically. If you are running in a self-hosted Pachyderm cluster, then you will have to install the Pachyderm client and connect to it before `pachctl` will run. For more information, see the [Getting Started](https://docs.pachyderm.com/latest/getting_started/local_installation/) docs. 

Let's make sure that we're connected to the Pachyderm cluster by checking the version. 

(`pachctl` is the version of the client running locally, `pachd` is the version of the Pachyderm server running in the cluster) 

In [1]:
!pachctl version

COMPONENT           VERSION             
pachctl             2.0.0-beta.8        
pachd               2.0.0-beta.7        


We can always see the help to understand how a particular `pachctl` command works by adding the `--help` flag. 

In [2]:
!pachctl --help

Access the Pachyderm API.

Environment variables:
  PACH_CONFIG=<path>, the path where pachctl will attempt to load your config.
  JAEGER_ENDPOINT=<host>:<port>, the Jaeger server to connect to, if PACH_TRACE
    is set
  PACH_TRACE={true,false}, if true, and JAEGER_ENDPOINT is set, attach a Jaeger
    trace to any outgoing RPCs.
  PACH_TRACE_DURATION=<duration>, the amount of time for which PPS should trace
    a pipeline after 'pachctl create-pipeline' (PACH_TRACE must also be set).

Usage:
  pachctl [command]

Administration Commands:
  auth         Auth commands manage access to data in a Pachyderm cluster
  enterprise   Enterprise commands enable Pachyderm Enterprise features
  idp          Commands to manage identity provider integrations

Commands by Action:
  copy         Copy a Pachyderm resource.
  create       Create a new instance of a Pachyderm resource.
  delete       Delete an existing Pachyderm resource.
  diff         Show the differences between two Pachyderm resource

In [3]:
!pachctl create repo --help

Create a new repo.

Usage:
  pachctl create repo <repo> [flags]

Flags:
  -d, --description string   A description of the repo.
  -h, --help                 help for repo

Global Flags:
      --no-color   Turn off colors.
  -v, --verbose    Output verbose logs


## Pachyderm Data Repositories

Pachyderm organizes data into data repositories. This is somewhat similar to git as we'll see, but scales much better for all file types, such as images, machine learning models, csv files, and many others.

Let's first start by creating a data repository. 

### Create a data repo

In [4]:
!pachctl create repo data

In [5]:
!pachctl list repo

NAME CREATED      SIZE (MASTER) ACCESS LEVEL 
data 1 second ago ≤ 0B          [repoOwner]   


A data repository, similar to a git repository, will be what we use to organize and reference data. 

We can also view and explore our data repository in the Pachyderm [Console](https://docs.pachyderm.com/2.0.x-beta/getting_started/beginner_tutorial/#exploring-your-dag-in-pachyderm-console), which should look something like the following.

<img src="images/console_repo.png" alt="Drawing" style="width: 300px;"/>



When we list our repos, we can see that we have an empty data repository, so let's add some data.

### Add data

First, we'll create a small csv file locally with some of the iris data. 

In [6]:
%%writefile iris.csv
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
7.0,3.2,4.7,1.4,Iris-versicolor
6.4,3.2,4.5,1.5,Iris-versicolor
6.9,3.1,4.9,1.5,Iris-versicolor
5.5,2.3,4.0,1.3,Iris-versicolor
6.3,3.3,6.0,2.5,Iris-virginica
5.8,2.7,5.1,1.9,Iris-virginica
7.1,3.0,5.9,2.1,Iris-virginica
6.3,2.9,5.6,1.8,Iris-virginica

Writing iris.csv


Data repositories in Pachyderm automatically track versions of the data placed in them. Similar to Git, we organize our data via branches, so we will push our data to the master branch of our data repository.

In [7]:
!pachctl put file data@master -f iris.csv

iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s
iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s
iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s
iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s


We can look at the data that's been uploaded to our data repository, by listing the files on the master branch.

In [8]:
!pachctl list file data@master

NAME      TYPE SIZE 
/iris.csv file 364B 


### Delete data

Similarly, if we want to delete our file, we can do that as well. 

In [9]:
!pachctl delete file data@master:/iris.csv

In [10]:
!pachctl list file data@master

NAME TYPE SIZE 


Now, if we add it back again...

In [11]:
!pachctl put file data@master -f iris.csv

iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s
iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s
iris.csv 364.00 b / 364.00 b [=====================================] 0s 0.00 b/s


In [12]:
!pachctl list file data@master

NAME      TYPE SIZE 
/iris.csv file 364B 


No surprise, our file is there again. But when we list all of the commits that have been made to our repository, we can see the history of data on the master branch.

### Data commits

In [13]:
!pachctl list commit data

REPO BRANCH COMMIT                           FINISHED       SIZE ORIGIN DESCRIPTION
data master 6b01b8466d0c4f64b95c89ecc0905ad9 3 seconds ago  364B USER    
data master 19f08ef5d17f431e84ebfad149d403fb 4 seconds ago  0B   USER    
data master f5666e807bd340498a65129d77d496e6 10 seconds ago 364B USER    


Pachyderm keeps a record of all the changes that happen to the data repository. This way if we ever want to revert to a previous version of our data repository (dataset in this case), we can do it.

For example, if we wanted to go back in time to the first file we added, we can move the "head" of our master branch to the first commit. To do this, we run the following 

**Note:** the commit hashes will be different. Copy and past the hash(es) above to run it yourself.

In [14]:
!pachctl create branch data@master --head f5666e807bd340498a65129d77d496e6

In [15]:
!pachctl list branch data

BRANCH HEAD                             TRIGGER 
master f5666e807bd340498a65129d77d496e6 -       


As we can see when we list the history of our branch, we now only see the first commit (the head of our master branch). 

Let's go back to our most recent commit. 

In [16]:
!pachctl create branch data@master --head 6b01b8466d0c4f64b95c89ecc0905ad9

### Awesome Pachyderm Feature - Efficient Storage! 

If we list our repo info again, we can see that the *entire size* of the repo is just as big as original file, even though we added it a second time! Pachyderm is really smart in how it handles data. It can understand when the content of a file is a duplicate of something it's seen before to minimize the amount of storage needed. 

This means it's much, much cheaper to store and version data in Pachyderm than any other platform. 

Note: Deduplication happens per chunk (e.g. 8MBs per chunk), not per file. For more information on why this is better, see [this blog](https://medium.com/@jdoliner/debunking-the-fud-about-data-version-control-implementations-55cbe72014fb) on content-based chunking. 

(This feature was introduced in Pachyderm 2.0)

In [21]:
!pachctl list repo

NAME CREATED            SIZE (MASTER) ACCESS LEVEL 
data About a minute ago ≤ 364B        [repoOwner]   


## Pachyderm Pipelines

Managing and versioning data by itself is only half the story. Once you have data, you typically want to do something with it, whether it's transform it, run tests on it, or even train a model. 

**A Pachyderm Pipeline is how you apply code to your data.**

Pipelines work seemlessly with data inside your data repositories, but even better, these pipelines can be triggered by your data! 

This means that we can deploy a pipeline to transform the data from our `data` repo, and anytime we modify our data, the pipeline will automatically re-run. 

Initially, this can be a hard concept to grasp, so let's walk through an example.

### Count Pipeline

Let's say we just want to count the number of lines in our csv file. We can create a Pachyderm Pipeline that looks like the `yaml` below that uses a shell command to count the number of lines (we'll see why we use shell later on).

In [22]:
%%writefile count.yaml
pipeline:
    name: 'count'
description: 'Count the number of lines in a csv file'
input:
    pfs:
        repo: 'data'
        branch: 'master'
        glob: '/'
transform:
    image: alpine:3.14.0
    cmd: ['/bin/sh']
    stdin: ['wc -l /pfs/data/iris.csv > /pfs/out/line_count.txt']

Writing count.yaml


### Pipelines in detail

Let's break this pipeline down section by section and explain it: 

Every pipeline must have a unique name. In our case, we will call this one `count`. It's also good practice to give our pipeline a description to help others know what it does. 

When the pipeline runs, it will also **create a data repository** `count` for any files created when the pipeline runs. 
```yaml
pipeline:
  name: count
description: Count the number of lines in a csv file
```

The `input` section defines what Pachyderm Data Repositories (or other type of input) will be connected to the pipeline. In our case, the `master` branch of our `data` repo will be used. 

When the pipeline runs, it will map the files from the `master` branch of our `data` repo, into the file system at `/pfs/data/` (`/pfs/` stands for Pachyderm File System). 

We'll talk more about glob patterns in another tutorial, but in this example, `/` means that every file on the head commit of the master branch is accessible to the the pipeline. 

```yaml
input:
  pfs:
    repo: data
    branch: master
    glob: /
```

The `transform` portion of the pipeline defines what code should be run when the pipeline executes. Pachyderm Pipelines use Docker containers to allow code written in any language to be executed as a pipeline. In this case, we are using a Docker container `alpine:3.14.0` as our Docker image. When this pipeline runs, it execute the `cmd` along with the `stdin` inside our container. 

Our `stdin` command, will count the number of lines in `/pfs/data/iris.csv` and write the output to `/pfs/out/line_count.txt`. `/pfs/out` is a special location in Pachyderm pipelines. Anything written to this directory will be *commited* to the `count` data repository (automatically created) as the output of the pipeline.

```yaml
transform:
  image: alpine:3.14.0
  cmd: ['/bin/sh']
  stdin: ['wc -l /pfs/data/iris.csv > /pfs/out/line_count.txt']
```

### Creating pipelines

We can submit our pipeline to Pachyderm by using the `create pipeline` command.

We can also view our pipelines in the Pachyderm Console as well. Notice it automatically creates the output data repository with the same name. 

<img src="images/console_pipeline.png" alt="Drawing" style="width: 800px;"/>


In [23]:
!pachctl create pipeline -f count.yaml

### Monitor pipelines

If we list our pipelines, we can see the status of them. 

In [26]:
!pachctl list pipeline

NAME  VERSION INPUT  CREATED        STATE / LAST JOB  DESCRIPTION                             
count 1       data:/ 27 seconds ago running / success Count the number of lines in a csv file 


It looks like our pipeline is `running` and the last job succeeded. Let's take a look at the job.

A job is an execution of our pipeline. We can see our job status by running: 

In [27]:
!pachctl list job

ID                               SUBJOBS PROGRESS CREATED        MODIFIED
e4ec864907de4e76973b4ca2ff8b280e 1       ▇▇▇▇▇▇▇▇ 29 seconds ago 29 seconds ago 


We can also see that we have a new data repository called `count` that holds the output of our pipeline. 

### View pipeline output commits

In [28]:
!pachctl list repo

NAME  CREATED        SIZE (MASTER) ACCESS LEVEL 
count 32 seconds ago ≤ 22B         [repoOwner]  Output repo for pipeline count. 
data  2 minutes ago  ≤ 364B        [repoOwner]                                  


In [29]:
!pachctl list file count@master

NAME            TYPE SIZE 
/line_count.txt file 22B  


Let's download the file created by our `count` pipeline and see what's in it. 

In [30]:
!pachctl get file count@master:/line_count.txt -o ./line_count.txt

./line_count.txt 0.00 b / 22.00 b [--------------------------------] 0s 0.00 b/s
./line_count.txt 22.00 b / 22.00 b [===============================] 0s 0.00 b/s
./line_count.txt 22.00 b / 22.00 b [===============================] 0s 0.00 b/s
./line_count.txt 22.00 b / 22.00 b [===============================] 0s 0.00 b/s


We can see that our output file correctly counted the number of lines in our csv file. 

In [31]:
# Output file
!cat line_count.txt

12 /pfs/data/iris.csv


In [32]:
# Original file
!wc -l iris.csv

      12 iris.csv


### Data-Driven Pipelines
If we recall, all of our pipelines in Pachyderm are data-driven. They are always ready to run whenever the data contained in an input repository changes. So let's do that. Let's update our iris data (this time with 24 lines). 

In [33]:
%%writefile iris_v2.csv
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
7.0,3.2,4.7,1.4,Iris-versicolor
6.4,3.2,4.5,1.5,Iris-versicolor
6.9,3.1,4.9,1.5,Iris-versicolor
5.5,2.3,4.0,1.3,Iris-versicolor
6.3,3.3,6.0,2.5,Iris-virginica
5.8,2.7,5.1,1.9,Iris-virginica
7.1,3.0,5.9,2.1,Iris-virginica
6.3,2.9,5.6,1.8,Iris-virginica
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
7.0,3.2,4.7,1.4,Iris-versicolor
6.4,3.2,4.5,1.5,Iris-versicolor
6.9,3.1,4.9,1.5,Iris-versicolor
5.5,2.3,4.0,1.3,Iris-versicolor
6.3,3.3,6.0,2.5,Iris-virginica
5.8,2.7,5.1,1.9,Iris-virginica
7.1,3.0,5.9,2.1,Iris-virginica
6.3,2.9,5.6,1.8,Iris-virginica

Writing iris_v2.csv


We'll overwrite our original file with the command: 

In [34]:
!pachctl put file data@master:iris.csv -f iris_v2.csv

iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s
iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s
iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s
iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s
iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s
iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s
iris_v2.csv 728.00 b / 728.00 b [==================================] 0s 0.00 b/s


In [35]:
!pachctl list file data@master

NAME      TYPE SIZE 
/iris.csv file 728B 


In [36]:
!pachctl list commit data@master

REPO BRANCH COMMIT                           FINISHED      SIZE ORIGIN DESCRIPTION
data master b59721ccd5e048efb7df545604cd80ed 6 seconds ago 728B USER    
data master 6b01b8466d0c4f64b95c89ecc0905ad9 2 minutes ago 364B USER    
data master 19f08ef5d17f431e84ebfad149d403fb 2 minutes ago 0B   USER    
data master f5666e807bd340498a65129d77d496e6 2 minutes ago 364B USER    


We have a new commit to our `data` repository, so let's see what's happened to our pipeline. 

In [37]:
!pachctl list job

ID                               SUBJOBS PROGRESS CREATED        MODIFIED
b59721ccd5e048efb7df545604cd80ed 1       ▇▇▇▇▇▇▇▇ 10 seconds ago 10 seconds ago 
e4ec864907de4e76973b4ca2ff8b280e 1       ▇▇▇▇▇▇▇▇ 57 seconds ago 57 seconds ago 


We have a new job that has just run. But remember, we only uploaded a file to our input repo. Pachyderm intelligently tells pipelines to run when their input data changes. If we look at the output of our `count` repository, we now see 2 commits. 

In [38]:
!pachctl list commit count@master

REPO  BRANCH COMMIT                           FINISHED       SIZE ORIGIN DESCRIPTION
count master b59721ccd5e048efb7df545604cd80ed 12 seconds ago 22B  AUTO    
count master e4ec864907de4e76973b4ca2ff8b280e 52 seconds ago 22B  AUTO    


In [39]:
!pachctl get file count@master:/line_count.txt -o ./line_count_v2.txt

./line_count_v2.txt 22.00 b / 22.00 b [============================] 0s 0.00 b/s
./line_count_v2.txt 22.00 b / 22.00 b [============================] 0s 0.00 b/s
./line_count_v2.txt 22.00 b / 22.00 b [============================] 0s 0.00 b/s


In [40]:
!cat line_count_v2.txt

24 /pfs/data/iris.csv


### Awesome Pachyderm Feature - Data Lineage!

The data-driven nature of Pachyderm Pipelines allow you to reliably maintain data and process lineage at scale. Combining versioning data with code in Docker containers for pipelines, Pachyderm can be used to automate, debug, and maintain any data + code workflow. 

For example, if we want to know the lineage of our most recent `line_count.txt`, we can run: 

In [41]:
!pachctl list commit count@master

REPO  BRANCH COMMIT                           FINISHED           SIZE ORIGIN DESCRIPTION
count master b59721ccd5e048efb7df545604cd80ed 21 seconds ago     22B  AUTO    
count master e4ec864907de4e76973b4ca2ff8b280e About a minute ago 22B  AUTO    


This gives us the unique commit for that run of the `count` pipeline. We can use this commit to see the unique combination of inputs and pipelines that resulted in this file. 

In [42]:
!pachctl list commit b59721ccd5e048efb7df545604cd80ed

REPO       BRANCH COMMIT                           FINISHED       SIZE     ORIGIN DESCRIPTION
data       master b59721ccd5e048efb7df545604cd80ed 35 seconds ago 728B     USER    
count.spec master b59721ccd5e048efb7df545604cd80ed 35 seconds ago 241B     ALIAS   
count.meta master b59721ccd5e048efb7df545604cd80ed 30 seconds ago 1.338KiB AUTO    
count      master b59721ccd5e048efb7df545604cd80ed 31 seconds ago 22B      AUTO    


We will gloss over some details here, but the important thing is, we can see the commit to the `data` repo was initiated by a `USER`. We can see exactly what commit triggered the pipeline. 

In [43]:
!pachctl list file data@b59721ccd5e048efb7df545604cd80ed

NAME      TYPE SIZE 
/iris.csv file 728B 


If we inspect the job associated with this commit, then we can get all the information about what pipeline was run on the data from this commit.

In [44]:
!pachctl inspect job count@b59721ccd5e048efb7df545604cd80ed

ID: b59721ccd5e048efb7df545604cd80ed
Pipeline: count
Started: 44 seconds ago 
Duration: 3 seconds 
State: success
Reason: 
Processed: 1
Failed: 0
Skipped: 0
Recovered: 0
Total: 1
Data Downloaded: 728B
Data Uploaded: 22B
Download Time: Less than a second
Process Time: Less than a second
Upload Time: Less than a second
Datum Timeout: (duration: nil Duration)
Job Timeout: (duration: nil Duration)
Worker Status:
WORKER              JOB                 DATUM               STARTED             
Restarts: 0
ParallelismSpec: <nil>



Input:
{
  "pfs": {
    "name": "data",
    "repo": "data",
    "repo_type": "user",
    "branch": "master",
    "commit": "b59721ccd5e048efb7df545604cd80ed",
    "glob": "/"
  }
}

Transform:
{
  "image": "alpine:3.14.0",
  "cmd": [
    "/bin/sh"
  ],
  "stdin": [
    "wc -l /pfs/data/iris.csv > /pfs/out/line_count.txt"
  ]
} 
Output Commit: b59721ccd5e048efb7df545604cd80ed 


## Conclusion

Pachyderm is an incredibly powerful platform, providing the data foundation for machine learning and other data-driven workflows. In this tutorial, we walked through some of the basic concepts that will get you started with Pachyderm. 


Be sure to check out our [examples](https://github.com/pachyderm/examples) for use-case specific tutorials or [join our community on Slack](https://www.pachyderm.com/slack/) to learn how to apply Pachyderm to your use-case.

See our [documentation](https://docs.pachyderm.com/latest/) for more in-depth explainations on the topics covered here. 